<a href="https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Study 1: Designing explainable speech-based machine learning models of depression

To open this ipynb in Colab, click the "Open in Colab" button at the top of the ipynb in Github, or [follow this link](https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/main/main.ipynb).

Given that Colab doesn't automatically load any of the content (data or other functions) from the Github repo, running the code below will copy the repo into the workspace directory for use. To save this ipynb file back to Github, select **File > Save** (which should show the repo if you're signed in) or **File > Save a copy in Github** if it's in the menu.

Note that the content of the data files or any of the other file structures are not saved back to Github, so make sure that if you make changes to things there, that you put them in Github separately.

In [1]:
# Clone Github Repo into the temporary local environment so data can be accessed and manipulated
!git clone https://github.com/cgenevier/CSCI5622-HW4.git
%cd CSCI5622-HW4

Cloning into 'CSCI5622-HW4'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 403 (delta 8), reused 393 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (403/403), 5.14 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/CSCI5622-HW4


In [3]:
# Import necessary libraries

# Helpers
import glob

# Pandas, seaborn, and numpy for data manipulation
import pandas as pd
pd.set_option("display.max_rows", None)
import statistics as stat
import seaborn as sns
import numpy as np
np.random.seed(42)

# Keras & TensorFlow for building the neural networks
import itertools, json, time
from itertools import count
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks, backend as K
tf.random.set_seed(42)

# Feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# Matplotlib for graphing
import matplotlib.pyplot as plt

In [4]:
# Import Depression Labels
# Columns: Participant_ID, PHQ_Score
depression_labels = pd.read_csv("data/DepressionLabels.csv")

# Rename Participant_ID to ParticipantID to match accoustic files below & force trimmed string type
depression_labels = depression_labels.rename(columns={"Participant_ID": "ParticipantID"})
depression_labels["ParticipantID"] = depression_labels["ParticipantID"].astype(str).str.strip()

# Import Text Dataset (for text feature extraction)
# Note: When comparing the E-DAIC_Transcripts files to the corresponding E-DAIC Acoustics files,
# it looks like the transcripts sometimes only contain partial data from the accoustics text column -
# for example, 386_Transcript.csv - so it seems to make sense to concatenate Text data in the
# Acoustics file for completeness.
rows = []
for p in glob.glob("data/E-DAIC_Acoustics/*_utterance_agg.csv"):
    df = pd.read_csv(p)
    df["ParticipantID"] = df["ParticipantID"].astype(str).str.strip()
    full_text = " ".join(df["Text"].dropna().astype(str))
    full_text = " ".join(full_text.split())  # collapse whitespace
    rows.append({"ParticipantID": df["ParticipantID"].iloc[0], "FullText": full_text})

# Columns: ParticipantID, FullText
text_df = pd.DataFrame(rows)
# Merge with labels. Columns: ParticipantID, FullText, PHQ_Score
lang_df = depression_labels.merge(text_df, on="ParticipantID", how="inner")


# Import Accoustic Dataset (for part c, d)
# Note: May need to aggregate the data per participant, ex: mean, std, iqr for each feature
# @TODO


### (a) (2 points) Extracting language features.

**Syntactic vectorizers:** count vectorizer (e.g., CountVectorizer from sklearn) transforming
a collection of text documents into a numerical matrix of word or token counts; TFIDF vectorizer (e.g., TfidfVectorizer from sklearn) incorporating document-level weighting,
which emphasizes words significant to specific documents’ part-of-speech features counting
the distribution of part of speech tags over a document

In [5]:
# Use TfidfVectorizer from sklearn
vect = TfidfVectorizer(max_features=1000)
X_tfidf = vect.fit_transform(lang_df["FullText"])

# Convert sparse matrix to DataFrame
tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vect.get_feature_names_out()
)

# Add ParticipantID column & move to first column
tfidf_df["ParticipantID"] = lang_df["ParticipantID"].values
cols = ["ParticipantID"] + [c for c in tfidf_df.columns if c != "ParticipantID"]
tfidf_df = tfidf_df[cols]

# Add back in PHQ_Score & move to second column
tfidf_df = tfidf_df.merge(depression_labels, on="ParticipantID", how="inner")
cols = ["ParticipantID", "PHQ_Score"] + [c for c in tfidf_df.columns if c not in ["ParticipantID", "PHQ_Score"]]
tfidf_df = tfidf_df[cols]

# Inspect dataframe
tfidf_df.head()

,ParticipantID,PHQ_Score,10,12,15,16,18,19,20,30,...,years,yes,yesterday,yet,york,you,young,younger,your,yourself
0,386,11,0.008629,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.017304,0.010626,0.000000,0.000000,0.000000,0.352579,0.011235,0.0,0.016919,0.019328
1,387,2,0.027755,0.000000,0.0,0.0,0.0,0.0,0.032038,0.0,...,0.074209,0.000000,0.000000,0.000000,0.047486,0.346512,0.072275,0.0,0.108838,0.031083
2,388,17,0.031186,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.041692,0.038404,0.000000,0.040133,0.000000,0.283166,0.000000,0.0,0.020382,0.034926
3,389,14,0.054573,0.052964,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.036479,0.000000,0.000000,0.000000,0.000000,0.325182,0.000000,0.0,0.089168,0.000000
4,390,9,0.000000,0.000000,0.0,0.0,0.0,0.0,0.016775,0.0,...,0.067999,0.008948,0.020119,0.000000,0.000000,0.123706,0.000000,0.0,0.009498,0.016275


**Semantic features:** sentiment scores (e.g., Vader, https://github.com/cjhutto/vaderSentiment),
topic distribution (using topic modeling), or named entities

**Advanced features:** word embeddings, such as Word2Vec or BERT (e.g., pytorch-pretrainedbert)) for capturing contextual meaning

### (b) (2 points) Estimating depression severity with interpretable models using language features.

### (c) (2 points) Estimating depression severity with interpretable models using acoustic features.

### (d) (2 points) Estimating depression severity with unimodal and multimodal deep learning models.

### (e) (2 points) Explainable ML.

### (f) (Bonus, 2 points) Experimenting with transformers.